<a href="https://colab.research.google.com/github/snow-white2024/AIFFEL_quest_cr/blob/master/250408_%EC%8B%A4%EC%A0%9C%EC%9D%8C%EC%84%B1%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A6%9D%EA%B0%95%ED%9B%84%ED%95%99%EC%8A%B5.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
dataset_path = '/content/drive/MyDrive/dataset'

In [3]:
import librosa
import numpy as np
import os

def extract_features(file_path):
    # 오디오 파일 로드
    audio, sample_rate = librosa.load(file_path, sr=None)

    # MFCC 특징 추출
    mfcc = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=13)

    # MFCC의 평균값을 특징으로 사용
    mfcc_mean = np.mean(mfcc, axis=1)

    return mfcc_mean

def load_data(dataset_path):
    features = []
    labels = []

    stable_path = os.path.join(dataset_path, 'stable')  # 안정된 상태 폴더
    for filename in os.listdir(stable_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(stable_path, filename)
            feature = extract_features(file_path)
            features.append(feature)
            labels.append(0)  # 안정된 상태 레이블은 0

    running_path = os.path.join(dataset_path, 'running')  # 중고강도 상태 폴더
    for filename in os.listdir(running_path):
        if filename.endswith('.wav'):
            file_path = os.path.join(running_path, filename)
            feature = extract_features(file_path)
            features.append(feature)
            labels.append(1)  # 중고강도 상태 레이블은 1

    return np.array(features), np.array(labels)

# 구글 드라이브의 경로로 데이터 로드
dataset_path = '/content/drive/MyDrive/dataset'  # 경로를 실제 위치에 맞게 변경하세요
X, y = load_data(dataset_path)


In [4]:
import librosa
import numpy as np

# 피치 변형 함수
def pitch_shift(audio, sample_rate, n_steps):
    return librosa.effects.pitch_shift(audio, sample_rate, n_steps=n_steps)

# 시간 신속도 조정 함수
def time_stretch(audio, rate):
    return librosa.effects.time_stretch(audio, rate)

# 데이터 증강 함수
def augment_data(audio, sample_rate):
    augmented_data = []

    # 원본 데이터
    augmented_data.append(audio)

    # 피치 변형
    augmented_data.append(pitch_shift(audio, sample_rate, n_steps=2))  # +2피치
    augmented_data.append(pitch_shift(audio, sample_rate, n_steps=-2))  # -2피치

    # 시간 신속도 조정
    augmented_data.append(time_stretch(audio, rate=1.2))  # 느리게
    augmented_data.append(time_stretch(audio, rate=0.8))  # 빠르게

    return augmented_data

# 예시: 오디오 파일을 로드하고 증강 적용
def load_and_augment(file_path):
    audio, sample_rate = librosa.load(file_path, sr=None)
    augmented_audios = augment_data(audio, sample_rate)
    return augmented_audios


In [6]:
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical

# 레이블을 one-hot 인코딩
y = to_categorical(y, num_classes=2)

# 데이터셋을 학습용과 테스트용으로 나누기
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Dropout

# LSTM 모델 구축
model = Sequential()
model.add(LSTM(128, input_shape=(X_train.shape[1], 1), return_sequences=True))
model.add(Dropout(0.5))
model.add(LSTM(64, return_sequences=False))
model.add(Dropout(0.5))
model.add(Dense(2, activation='softmax'))  # 두 가지 클래스를 예측

# 모델 컴파일
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


/usr/local/lib/python3.11/dist-packages/keras/src/layers/rnn/rnn.py:200: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [8]:
# 데이터 차원 맞추기 (LSTM에 맞게 입력 형태 변환)
X_train = np.expand_dims(X_train, axis=-1)
X_test = np.expand_dims(X_test, axis=-1)


In [9]:
# 모델 학습
model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.5000 - loss: 0.7118 - val_accuracy: 1.0000 - val_loss: 0.6611
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 407ms/step - accuracy: 0.3333 - loss: 0.7025 - val_accuracy: 1.0000 - val_loss: 0.6364
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step - accuracy: 0.8333 - loss: 0.6622 - val_accuracy: 1.0000 - val_loss: 0.6118
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 307ms/step - accuracy: 0.6667 - loss: 0.6217 - val_accuracy: 1.0000 - val_loss: 0.5853
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 187ms/step - accuracy: 0.8333 - loss: 0.5733 - val_accuracy: 1.0000 - val_loss: 0.5616
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 299ms/step - accuracy: 1.0000 - loss: 0.5172 - val_accuracy: 1.0000 - val_loss: 0.5359
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 191ms/step - accuracy: 1.0000 - loss: 0.5563 - val_accuracy: 1.0000 - val_loss: 0.5080
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 221ms/step - accuracy: 0.6667 - loss: 0.6275 - val_accuracy: 1.0000 - val_loss: 0.

In [10]:
# 모델 평가
loss, accuracy = model.evaluate(X_test, y_test)
print(f'Test accuracy: {accuracy}')


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 209ms/step - accuracy: 1.0000 - loss: 0.0461
Test accuracy: 1.0
